### UPS

In [ ]:
%matplotlib inline
import sys
from pathlib import Path

import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import ticker
from matplotlib.colorbar import Colorbar
from matplotlib.colors import ListedColormap


In [ ]:
# Ensure custom module path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# 颜色设定
colors = list(tol_cset("vibrant"))
if r"sunset" not in plt.colormaps():
    plt.colormaps.register(tol_cmap("sunset"))
if r"rainbow_PuRd" not in plt.colormaps():
    plt.colormaps.register(tol_cmap("rainbow_PuRd"))  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

### UPS: 不同状态下的电极

In [ ]:
def extract_name(path: Path):
    parts = path.parts
    # 找出所有含有 "+" 的文件夹名, 注意还有一个 UPS+XPS
    idx = [i for i, p in enumerate(parts) if "+" in p][1]
    return f"{parts[idx - 1]}", f"{parts[idx]}"

In [ ]:
# 读取数据
path_ups = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XPS+UPS\ExSitu\αMnO2").glob(
        r"**\Data\*_UPS.txt"
    )
)
# 应用于列表
sample_names = [extract_name(p) for p in path_ups]
# 读取数据
df_ups = [
    pd.read_csv(
        p,
        sep="\s+",
        skiprows=7,
        index_col=None,
        names=[r"B.E.1", r"Overview:vb", r"B.E.2", r"Overview:wf-10v"],
        header=None,
        comment="#",
    )
    for p in path_ups
]  # type: ignore

In [ ]:
# 导出所有的 UPS 数据
df = pd.concat(df_ups, axis=1, ignore_index=True)
new_columns = []
for i, name in enumerate(sample_names):
    base_idx = i * 4
    new_columns += [r"B.E.1", f"{name[0]}_{name[1]}_vb", r"B.E.2", f"{name[0]}_{name[1]}_wf-10v"]
df.columns = new_columns
df.to_csv(path_out.joinpath(r"UPS_all.csv"), index=False, sep=",", header=True, float_format="%.3f")

In [ ]:
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

colors_ups = cm.tab20.colors  # type: ignore
for i, (name, ups) in enumerate(zip(sample_names, df_ups)):
    ax.plot(ups["B.E.1"], ups["Overview:vb"], c=colors_ups[i], label=f"{name[0]} in {name[1]}", zorder=10 - i)

ax.set_xlabel(r"Binding Energy (eV)", fontsize=11)
ax.set_xlim(20, -10)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5))

ax.set_ylabel(r"Intensity (counts)", fontsize=11)
ax.set_ylim(-1000, 400000)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=100000, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=50000, offset=0))
formatter = ticker.ScalarFormatter()
formatter.set_powerlimits((-2, 2))  # 控制何时使用科学计数法的阈值
formatter.set_scientific(True)
ax.yaxis.set_major_formatter(formatter)
# ax.legend(loc='best', ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.15, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

colors_ups = cm.tab20.colors  # type: ignore
for i, (name, ups) in enumerate(zip(sample_names, df_ups)):
    ax.plot(ups["B.E.2"], ups["Overview:wf-10v"], c=colors_ups[i], label=f"{name[0]} in {name[1]}", zorder=10 - i)

ax.set_xlabel(r"Binding Energy (eV)", fontsize=11)
ax.set_xlim(20, -10)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5))

ax.set_ylabel(r"Intensity (counts)", fontsize=11)
ax.set_ylim(-1000, 600000)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=100000, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=50000, offset=0))
formatter = ticker.ScalarFormatter()
formatter.set_powerlimits((-2, 2))  # 控制何时使用科学计数法的阈值
formatter.set_scientific(True)
ax.yaxis.set_major_formatter(formatter)
# ax.legend(loc='best', ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(1.03, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.9,
    r"Bias - 10 eV",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=13,
    color="k",
)

plt.savefig(
    Path.joinpath(path_out, r"300_UPS_all.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"600_UPS_all.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.show()